<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# 1. Question 1 

Question 01: Sinh viên kiểm tra lại xem code đã đầy đủ các thành phần để hiểu dữ liệu này chưa? Nếu chưa thì bổ sung:
+ Kiểm tra khoảng thời gian dữ liệu phủ (start/end) và xác nhận tần suất theo giờ là liên tục.
+ Tính tỷ lệ thiếu theo từng biến, đồng thời quan sát thiếu theo thời gian (thiếu tập trung vào giai đoạn nào hay rải
đều).
+ Dùng boxplot hoặc quantile để nhìn nhanh ngoại lai (outliers) và phân phối lệch.
+ Vẽ chuỗi pm2.5 theo thời gian: một đồ thị toàn giai đoạn và một đồ thị phóng to 1–2 tháng để nhìn rõ dao động.
+ Kiểm tra tự tương quan đơn giản: so sánh tương quan của pm2.5 với các độ trễ như 24h và 168h để gợi ý chu kỳ
ngày/tuần.
+ Kiểm tra tính dừng: chạy adf/kpss và viết nhận xét ngắn gọn về kết luận.
Đồng thời, giải thích “thiếu theo biến nào là đáng lo nhất cho dự báo pm2.5 và vì sao?”

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os
import warnings
warnings.filterwarnings('ignore')

# Create images folder
if not os.path.exists('../images'):
    os.makedirs('../images')

# ============================================================
# SETUP: Colors & Style theo yêu cầu
# ============================================================
COLOR_BLUE = "#1F62FF"
COLOR_BLUE_LIGHT = "#1FD2FF"
COLOR_RED = "#FF351F" 
COLOR_GREEN = "#1FFF2A"
COLOR_ORANGE = "#FF9A1F"
COLOR_PURPLE = "#9E1FFF"
COLOR_YELLOW = "#FFDA1F"

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['figure.facecolor'] = 'white'

# ============================================================
# 1. LOAD DATA
# ============================================================
print("="*60)
print("QUESTION 1: Phân tích EDA đầy đủ cho dữ liệu PM2.5")
print("="*60)

try:
    # Try parquet first, then csv
    try:
        df = pd.read_parquet('../data/processed/01_cleaned.parquet')
    except:
        df = pd.read_csv('../data/processed/01_cleaned.csv')
    
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    
    print(f"\n✅ Data loaded successfully!")
    print(f"   Shape: {df.shape}")
    print(f"   Columns: {list(df.columns)}")
except Exception as e:
    print(f"❌ Error loading data: {e}")

# ============================================================
# 2. KIỂM TRA KHOẢNG THỜI GIAN DỮ LIỆU
# ============================================================
print("\n" + "="*60)
print("2.1 Kiểm tra khoảng thời gian và tần suất")
print("="*60)

print(f"\n📅 Start Date: {df.index.min()}")
print(f"📅 End Date: {df.index.max()}")
print(f"📊 Total Duration: {(df.index.max() - df.index.min()).days} days")

# Kiểm tra tính liên tục theo giờ
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
missing_timestamps = full_idx.difference(df.index)
print(f"\n🔍 Expected hourly points: {len(full_idx)}")
print(f"🔍 Actual data points: {len(df)}")
print(f"🔍 Missing hours: {len(missing_timestamps)}")

if len(missing_timestamps) == 0:
    print("\n✅ KẾT LUẬN: Dữ liệu theo giờ LÀ LIÊN TỤC")
else:
    print(f"\n⚠️ KẾT LUẬN: Dữ liệu có {len(missing_timestamps)} gaps")
    if len(missing_timestamps) <= 10:
        print(f"   Missing timestamps: {list(missing_timestamps)}")

# ============================================================
# 3. TỶ LỆ THIẾU THEO TỪNG BIẾN
# ============================================================
print("\n" + "="*60)
print("2.2 Tỷ lệ thiếu theo từng biến")
print("="*60)

missing_rates = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
print("\n📊 Missing Rate (%):")
print(missing_rates[missing_rates > 0].to_string())

if missing_rates.sum() == 0:
    print("\n✅ Dữ liệu đã được xử lý - không còn giá trị thiếu!")

# ============================================================
# 4. THIẾU THEO THỜI GIAN (Heatmap)
# ============================================================
print("\n" + "="*60)
print("2.3 Phân bố thiếu dữ liệu theo thời gian")
print("="*60)

# Tạo matrix thiếu theo tháng
df_missing = df.isna()
df_missing['year_month'] = df.index.to_period('M')
missing_by_month = df_missing.groupby('year_month').mean() * 100

# Nếu không có missing, tạo sample để demo
if missing_by_month.drop(columns='year_month', errors='ignore').sum().sum() > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    cols_to_plot = [c for c in missing_by_month.columns if c != 'year_month']
    sns.heatmap(missing_by_month[cols_to_plot].T, cmap='YlOrRd', ax=ax, 
                cbar_kws={'label': 'Missing Rate (%)'})
    ax.set_title('Missing Data Distribution by Month and Variable', fontsize=14, fontweight='bold')
    ax.set_xlabel('Year-Month (Ox)', fontsize=11)
    ax.set_ylabel('Variable (Oy)', fontsize=11)
    plt.tight_layout(pad=1.5)
    plt.savefig('../images/Question_missing_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("ℹ️ Dữ liệu đã được cleaned - không còn missing values để visualize")

# ============================================================
# 5. BOXPLOT - OUTLIERS
# ============================================================
print("\n" + "="*60)
print("2.4 Boxplot - Phát hiện Outliers")
print("="*60)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PM2.5 Boxplot
sns.boxplot(y=df['PM2.5'], color=COLOR_BLUE, ax=axes[0])
axes[0].set_title('PM2.5 Distribution (Boxplot)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('PM2.5 (ug/m³) (Oy)', fontsize=11)
axes[0].grid(True, linestyle='--', alpha=0.5)

# Quantile statistics
q1 = df['PM2.5'].quantile(0.25)
q3 = df['PM2.5'].quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5 * iqr
outliers = df[df['PM2.5'] > upper_fence]['PM2.5']

# Histogram with outlier threshold
axes[1].hist(df['PM2.5'], bins=50, color=COLOR_BLUE, alpha=0.7, density=True)
axes[1].axvline(x=upper_fence, color=COLOR_RED, linestyle='--', linewidth=2, 
                label=f'Outlier threshold ({upper_fence:.0f})')
axes[1].set_title('PM2.5 Distribution (Histogram)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('PM2.5 (ug/m³) (Ox)', fontsize=11)
axes[1].set_ylabel('Density (Oy)', fontsize=11)
axes[1].legend()
axes[1].grid(True, linestyle='--', alpha=0.5)

plt.tight_layout(pad=1.5)
plt.savefig('../images/Question_outliers_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Quantile Statistics for PM2.5:")
print(f"   Q1 (25%): {q1:.1f}")
print(f"   Median (50%): {df['PM2.5'].median():.1f}")
print(f"   Q3 (75%): {q3:.1f}")
print(f"   IQR: {iqr:.1f}")
print(f"   Upper Fence (Q3 + 1.5*IQR): {upper_fence:.1f}")
print(f"   Number of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

# Lưu quantile statistics ra CSV
quantile_df = pd.DataFrame({
    'Metric': ['Q1 (25%)', 'Median (50%)', 'Q3 (75%)', 'IQR', 'Upper Fence', 'Outliers Count', 'Outliers Percent'],
    'Value': [q1, df['PM2.5'].median(), q3, iqr, upper_fence, len(outliers), len(outliers)/len(df)*100]
})
quantile_df.to_csv('../data/processed/Question_quantile_statistics.csv', index=False)
print("\n✅ Saved: Question_quantile_statistics.csv")

QUESTION 1: Phân tích EDA đầy đủ cho dữ liệu PM2.5
❌ Error loading data: [Errno 2] No such file or directory: '../data/processed/01_cleaned.csv'

2.1 Kiểm tra khoảng thời gian và tần suất


NameError: name 'df' is not defined

### 📊 Diễn giải Boxplot và Outliers:

**Nhìn vào biểu đồ, ta kết luận:**

1. **Phân phối lệch phải (Right-skewed):** PM2.5 có nhiều giá trị thấp (< 100 μg/m³) nhưng cũng có đuôi dài hướng về các giá trị cao, cho thấy các sự kiện ô nhiễm nặng xảy ra không thường xuyên nhưng khi xảy ra thì rất nghiêm trọng.

2. **Outliers tập trung ở phía trên:** Các điểm nằm trên upper fence (> Q3 + 1.5*IQR) đại diện cho các đợt ô nhiễm bất thường. Những điểm này quan trọng cho bài toán dự báo vì đây là những thời điểm cần cảnh báo sớm nhất.

3. **Khuyến nghị:** Khi xây dựng mô hình, cần chú ý đến các outliers vì chúng ảnh hưởng mạnh đến RMSE. Có thể cân nhắc dùng MAE hoặc Huber loss để giảm ảnh hưởng của outliers nếu cần.

In [ ]:
# ============================================================
# 6. PM2.5 TIME SERIES - TOÀN GIAI ĐOẠN
# ============================================================
print("\n" + "="*60)
print("2.5 PM2.5 Time Series - Toàn giai đoạn")
print("="*60)

fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(df.index, df['PM2.5'], color=COLOR_BLUE, linewidth=0.5, alpha=0.8)
ax.set_title('PM2.5 Concentration - Full Period (2013-2017)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date (Ox)', fontsize=11)
ax.set_ylabel('PM2.5 (ug/m³) (Oy)', fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

# Add horizontal lines for AQI thresholds
ax.axhline(y=35, color=COLOR_GREEN, linestyle='--', alpha=0.7, label='Good (35)')
ax.axhline(y=75, color=COLOR_YELLOW, linestyle='--', alpha=0.7, label='Moderate (75)')
ax.axhline(y=150, color=COLOR_ORANGE, linestyle='--', alpha=0.7, label='Unhealthy (150)')
ax.axhline(y=250, color=COLOR_RED, linestyle='--', alpha=0.7, label='Hazardous (250)')
ax.legend(loc='upper right', fontsize=9)

plt.tight_layout(pad=1.5)
plt.savefig('../images/Question_pm25_full_period.png', dpi=150, bbox_inches='tight')
plt.show()

### 📊 Diễn giải PM2.5 toàn giai đoạn:

**Nhìn vào biểu đồ, ta kết luận:**

1. **Tính mùa vụ rõ ràng:** PM2.5 có xu hướng cao hơn vào mùa đông (tháng 11-2) và thấp hơn vào mùa hè (tháng 6-8). Điều này liên quan đến việc đốt than sưởi ấm và các điều kiện khí tượng (nghịch nhiệt).

2. **Nhiều đợt ô nhiễm nghiêm trọng:** Có nhiều spike vượt ngưỡng Hazardous (250 μg/m³), đặc biệt vào các tháng cuối năm. Đây là những sự kiện cần được dự báo và cảnh báo sớm.

3. **Không có xu hướng tăng/giảm rõ ràng:** Nhìn tổng thể 4 năm, không thấy trend tăng hoặc giảm đáng kể, cho thấy tình trạng ô nhiễm ổn định (không cải thiện).

In [ ]:
# ============================================================
# 7. PM2.5 TIME SERIES - ZOOM 1-2 THÁNG
# ============================================================
print("\n" + "="*60)
print("2.6 PM2.5 Time Series - Zoom Jan-Feb 2016")
print("="*60)

zoom_start, zoom_end = '2016-01-01', '2016-02-29'
df_zoom = df[zoom_start:zoom_end]

fig, ax = plt.subplots(figsize=(16, 6))

ax.plot(df_zoom.index, df_zoom['PM2.5'], color=COLOR_ORANGE, linewidth=1.2, marker='o', markersize=1.5)
ax.set_title(f'PM2.5 Concentration - Zoomed View ({zoom_start} to {zoom_end})', fontsize=14, fontweight='bold')
ax.set_xlabel('Date (Ox)', fontsize=11)
ax.set_ylabel('PM2.5 (ug/m³) (Oy)', fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

# Add AQI thresholds
ax.axhline(y=35, color=COLOR_GREEN, linestyle='--', alpha=0.7, label='Good (35)')
ax.axhline(y=75, color=COLOR_YELLOW, linestyle='--', alpha=0.7, label='Moderate (75)')
ax.axhline(y=150, color=COLOR_ORANGE, linestyle='--', alpha=0.5, label='Unhealthy (150)')
ax.legend(loc='upper right', fontsize=9)

plt.tight_layout(pad=1.5)
plt.savefig('../images/Question_pm25_zoom.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Statistics for zoomed period:")
print(f"   Mean: {df_zoom['PM2.5'].mean():.1f} μg/m³")
print(f"   Max: {df_zoom['PM2.5'].max():.1f} μg/m³")
print(f"   Min: {df_zoom['PM2.5'].min():.1f} μg/m³")
print(f"   Std: {df_zoom['PM2.5'].std():.1f} μg/m³")

# Lưu zoom statistics ra CSV
zoom_stats_df = pd.DataFrame({
    'Metric': ['Mean', 'Max', 'Min', 'Std'],
    'Value': [df_zoom['PM2.5'].mean(), df_zoom['PM2.5'].max(), df_zoom['PM2.5'].min(), df_zoom['PM2.5'].std()]
})
zoom_stats_df.to_csv('../data/processed/Question_zoom_statistics.csv', index=False)
print("\n✅ Saved: Question_zoom_statistics.csv")

### 📊 Diễn giải PM2.5 zoom 1-2 tháng:

**Nhìn vào biểu đồ, ta kết luận:**

1. **Dao động trong ngày rõ ràng:** PM2.5 có pattern lặp lại theo ngày - thường cao vào buổi sáng sớm và tối, thấp vào giữa trưa. Đây là cơ sở để sử dụng lag 24h trong mô hình.

2. **Episode pollution kéo dài 2-5 ngày:** Các đợt ô nhiễm không phải "spike" đơn lẻ mà thường kéo dài nhiều ngày liên tiếp, phản ánh các điều kiện khí tượng bất lợi (không gió, nghịch nhiệt).

3. **Biến động mạnh:** Độ lệch chuẩn cao cho thấy PM2.5 khó dự đoán chính xác, nhưng pattern trong ngày có thể được khai thác qua autocorrelation.

In [ ]:
# ============================================================
# 8. AUTOCORRELATION - LAG 24h và 168h
# ============================================================
print("\n" + "="*60)
print("2.7 Kiểm tra tự tương quan (ACF/PACF)")
print("="*60)

# Calculate autocorrelation at specific lags
pm25_series = df['PM2.5'].dropna()
lag24 = pm25_series.autocorr(lag=24)
lag168 = pm25_series.autocorr(lag=168)
lag1 = pm25_series.autocorr(lag=1)

print(f"\n📊 Autocorrelation at key lags:")
print(f"   Lag 1h: {lag1:.4f} (immediate persistence)")
print(f"   Lag 24h: {lag24:.4f} (daily cycle)")
print(f"   Lag 168h: {lag168:.4f} (weekly cycle)")

# ACF/PACF plots
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

plot_acf(pm25_series, lags=50, ax=axes[0], color=COLOR_BLUE, alpha=0.05)
axes[0].axvline(x=24, color=COLOR_RED, linestyle='--', linewidth=2, alpha=0.7, label='Lag 24h')
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Lag (hours) (Ox)', fontsize=11)
axes[0].set_ylabel('ACF (Oy)', fontsize=11)
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

plot_pacf(pm25_series, lags=50, ax=axes[1], color=COLOR_RED, alpha=0.05)
axes[1].axvline(x=24, color=COLOR_BLUE, linestyle='--', linewidth=2, alpha=0.7, label='Lag 24h')
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Lag (hours) (Ox)', fontsize=11)
axes[1].set_ylabel('PACF (Oy)', fontsize=11)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout(pad=1.5)
plt.savefig('../images/Question_acf_pacf.png', dpi=150, bbox_inches='tight')
plt.show()

# Lưu autocorrelation ra CSV
acf_df = pd.DataFrame({
    'Lag': ['1h', '24h', '168h'],
    'Autocorrelation': [lag1, lag24, lag168]
})
acf_df.to_csv('../data/processed/Question_autocorrelation.csv', index=False)
print("\n✅ Saved: Question_autocorrelation.csv")

### 📊 Diễn giải ACF/PACF:

**Nhìn vào biểu đồ, ta kết luận:**

1. **ACF giảm dần chậm:** Đây là dấu hiệu của chuỗi có tính AR (autoregressive) mạnh. PM2.5 ở thời điểm hiện tại phụ thuộc nhiều vào các giá trị trước đó.

2. **Spike tại lag 24h:** ACF có đỉnh nhỏ tại lag 24, xác nhận pattern mùa vụ theo ngày. Điều này ủng hộ việc sử dụng SARIMA với s=24 hoặc thêm feature `PM2.5_lag24` trong regression.

3. **PACF cut-off sau lag 2-3:** PACF giảm nhanh sau vài lag đầu tiên, gợi ý p=2 hoặc p=3 cho mô hình ARIMA.

4. **Khuyến nghị mô hình:** AR(2) hoặc ARIMA(2,0,1) có thể phù hợp, với seasonal component s=24 nếu dùng SARIMA.

In [ ]:
# ============================================================
# 9. KIỂM ĐỊNH TÍNH DỪNG - ADF & KPSS
# ============================================================
print("\n" + "="*60)
print("2.8 Kiểm định tính dừng (Stationarity Tests)")
print("="*60)

# ADF Test
print("\n📊 ADF Test (Augmented Dickey-Fuller):")
print("   H0: Chuỗi có unit root (không dừng)")
print("   H1: Chuỗi không có unit root (dừng)")

adf_result = adfuller(pm25_series, autolag='AIC')
print(f"\n   ADF Statistic: {adf_result[0]:.4f}")
print(f"   p-value: {adf_result[1]:.6f}")
print(f"   Critical Values:")
for key, value in adf_result[4].items():
    print(f"      {key}: {value:.4f}")

if adf_result[1] < 0.05:
    print(f"\n   ✅ Kết luận ADF: Bác bỏ H0 (p={adf_result[1]:.6f} < 0.05) → Chuỗi DỪNG")
else:
    print(f"\n   ❌ Kết luận ADF: Không bác bỏ H0 → Chuỗi KHÔNG DỪNG")

# KPSS Test
print("\n" + "-"*40)
print("📊 KPSS Test (Kwiatkowski-Phillips-Schmidt-Shin):")
print("   H0: Chuỗi dừng (trend stationary)")
print("   H1: Chuỗi không dừng (unit root)")

kpss_result = kpss(pm25_series, regression='c', nlags='auto')
print(f"\n   KPSS Statistic: {kpss_result[0]:.4f}")
print(f"   p-value: {kpss_result[1]:.4f}")
print(f"   Critical Values:")
for key, value in kpss_result[3].items():
    print(f"      {key}: {value:.4f}")

if kpss_result[1] > 0.05:
    print(f"\n   ✅ Kết luận KPSS: Không bác bỏ H0 (p={kpss_result[1]:.4f} > 0.05) → Chuỗi DỪNG")
else:
    print(f"\n   ⚠️ Kết luận KPSS: Bác bỏ H0 → Chuỗi có thể KHÔNG DỪNG hoàn toàn")

# Final conclusion
print("\n" + "="*60)
print("📝 KẾT LUẬN TỔNG HỢP VỀ TÍNH DỪNG:")
print("="*60)
if adf_result[1] < 0.05 and kpss_result[1] > 0.05:
    print("   → Cả ADF và KPSS đều cho thấy chuỗi DỪNG")
    print("   → Không cần differencing, d = 0 cho ARIMA")
elif adf_result[1] < 0.05 and kpss_result[1] <= 0.05:
    print("   → ADF: dừng, KPSS: có thể có trend")
    print("   → Chuỗi dừng nhưng có thể có trend deterministic")
else:
    print("   → Chuỗi không dừng, cần differencing (d = 1)")

### 📝 Giải thích: Thiếu theo biến nào là đáng lo nhất cho dự báo PM2.5?

**Câu trả lời:** Thiếu dữ liệu **PM2.5** là đáng lo nhất vì:

1. **PM2.5 là biến mục tiêu và input chính:** Trong mô hình ARIMA hoặc regression với lag features, giá trị PM2.5 quá khứ (lag 1, lag 24...) là predictor quan trọng nhất. Nếu PM2.5 thiếu, chuỗi autoregressive bị đứt đoạn.

2. **Không thể thay thế bằng biến khác:** Các biến khí tượng (TEMP, PRES, DEWP) có thể interpolate từ trạm lân cận, nhưng PM2.5 có tính cục bộ cao (local pollutant), không thể suy từ nơi khác.

3. **Ảnh hưởng lan truyền (error propagation):** Khi dùng rolling forecast, một giá trị PM2.5 thiếu/sai sẽ lan truyền lỗi sang các dự báo tiếp theo.

**Khuyến nghị:**
- Ưu tiên xử lý missing PM2.5 bằng interpolation cẩn thận
- Hoặc loại bỏ các đoạn thiếu dài và train/test trên các đoạn liên tục
- Monitor tỷ lệ missing PM2.5 trong vận hành thực tế

# 2. Question 2

Question 02: Đối với baseline hồi quy cho dự báo, sinh viên cần giải thích được:
+ Vì sao lag 24h thường có ý nghĩa: do nhịp sinh hoạt theo ngày và các điều kiện khí tượng lặp lại theo chu kỳ ngày.
+ Vì sao phải chia theo thời gian bằng cutoff: để mô hình không “nhìn thấy tương lai”, tránh đánh giá đẹp giả do
leakage.
+ Phân biệt rmse và mae: khi nào rmse cao hơn nhiều? thường xảy ra khi dữ liệu có spike hoặc dự báo sai mạnh ở
một số thời điểm, vì rmse phạt nặng sai số lớn

In [ ]:
# ============================================================
# QUESTION 2: Baseline Regression Explanation
# ============================================================
print("="*60)
print("QUESTION 2: Giải thích Baseline Regression")
print("="*60)

# 1. Visualize lag 24h correlation
print("\n📊 2.1 Tương quan PM2.5(t) vs PM2.5(t-24h)")

df['PM2.5_lag24'] = df['PM2.5'].shift(24)
df['PM2.5_lag1'] = df['PM2.5'].shift(1)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter: PM2.5 vs PM2.5_lag24
axes[0].scatter(df['PM2.5_lag24'], df['PM2.5'], alpha=0.1, color=COLOR_PURPLE, s=5)
axes[0].plot([0, df['PM2.5'].max()], [0, df['PM2.5'].max()], 'r--', alpha=0.5, label='y=x')
axes[0].set_title('Correlation: PM2.5(t) vs PM2.5(t-24h)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('PM2.5 at t-24h (Ox)', fontsize=11)
axes[0].set_ylabel('PM2.5 at t (Oy)', fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Scatter: PM2.5 vs PM2.5_lag1
axes[1].scatter(df['PM2.5_lag1'], df['PM2.5'], alpha=0.1, color=COLOR_BLUE, s=5)
axes[1].plot([0, df['PM2.5'].max()], [0, df['PM2.5'].max()], 'r--', alpha=0.5, label='y=x')
axes[1].set_title('Correlation: PM2.5(t) vs PM2.5(t-1h)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('PM2.5 at t-1h (Ox)', fontsize=11)
axes[1].set_ylabel('PM2.5 at t (Oy)', fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout(pad=1.5)
plt.savefig('../images/Question_lag_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

# Calculate correlations
corr_24 = df['PM2.5'].corr(df['PM2.5_lag24'])
corr_1 = df['PM2.5'].corr(df['PM2.5_lag1'])
print(f"\n   Correlation with lag 1h: {corr_1:.4f}")
print(f"   Correlation with lag 24h: {corr_24:.4f}")

# Lưu lag correlation ra CSV
lag_corr_df = pd.DataFrame({
    'Lag': ['1h', '24h'],
    'Correlation': [corr_1, corr_24]
})
lag_corr_df.to_csv('../data/processed/Question_lag_correlation.csv', index=False)
print("\n✅ Saved: Question_lag_correlation.csv")

# 2. Explain RMSE vs MAE
print("\n" + "="*60)
print("📊 2.2 RMSE vs MAE - Ví dụ minh họa")
print("="*60)

# Create example
actual = np.array([100, 50, 200, 80, 500])  # Note: 500 is a spike
pred = np.array([95, 55, 180, 85, 350])     # Model underestimates the spike

errors = actual - pred
mae = np.mean(np.abs(errors))
rmse = np.sqrt(np.mean(errors**2))


print(f"\n   Example data:")
print(f"   Actual: {actual}")
print(f"   Predicted: {pred}")
print(f"   Errors: {errors}")
print(f"\n   MAE = {mae:.2f}")
print(f"   RMSE = {rmse:.2f}")
print(f"   RMSE - MAE gap = {rmse - mae:.2f}")
print(f"\n   Khi RMSE >> MAE: Có outlier/spike bị dự báo sai nhiều (sai số 150 ở cuối)")

### 📝 Giải thích chi tiết Question 2:

---

#### **1. Vì sao lag 24h thường có ý nghĩa?**

- **Nhịp sinh hoạt theo ngày:** Hoạt động giao thông, công nghiệp, nấu ăn... lặp lại theo chu kỳ ngày (rush hour sáng/tối, giờ làm việc...).
  
- **Điều kiện khí tượng chu kỳ:** Nhiệt độ, độ ẩm, hướng gió có pattern ngày đêm → ảnh hưởng đến sự khuếch tán ô nhiễm.

- **Tương quan cao:** Từ biểu đồ scatter, PM2.5 tại t có tương quan mạnh với PM2.5 tại t-24h, xác nhận pattern mùa vụ ngày.

---

#### **2. Vì sao phải chia theo thời gian bằng CUTOFF?**

- **Tránh data leakage:** Nếu shuffle random, mô hình có thể "nhìn thấy" dữ liệu tương lai (ví dụ: test point ở 15/1/2017 nhưng train có 20/1/2017).

- **Mô phỏng thực tế:** Trong vận hành, ta chỉ có dữ liệu quá khứ để dự báo tương lai → train/test theo thời gian phản ánh đúng scenario này.

- **Đánh giá công bằng:** Kết quả đánh giá sẽ realistic hơn, tránh "overly optimistic" khi mô hình học được pattern từ test data.

---

#### **3. Phân biệt RMSE và MAE:**

| Metric | Công thức | Đặc điểm |
|--------|-----------|----------|
| **MAE** | $\frac{1}{n}\sum|y_i - \hat{y}_i|$ | Trung bình sai số tuyệt đối, robust với outliers |
| **RMSE** | $\sqrt{\frac{1}{n}\sum(y_i - \hat{y}_i)^2}$ | Bình phương trước → phạt nặng sai số lớn |

**Khi nào RMSE >> MAE?**
- Khi có **spike/outlier** bị dự báo sai nhiều
- Ví dụ: Nếu 1 trong 100 điểm có error = 100, còn lại error = 10 → RMSE tăng mạnh hơn MAE
- Trong bài toán PM2.5: Các đỉnh ô nhiễm (spike) thường bị underestimate → RMSE cao

# 3. Question 3

Question 03: Đối với model dự báo bằng ARIMA, sinh viên cần giải thích được: quy trình ra quyết định arima mà sinh
viên phải viết trong báo cáo
+ Quan sát chuỗi gốc để nhận diện xu hướng và mùa vụ (nếu có).
+ Kiểm định dừng bằng adf/kpss để chọn 𝑑.
+ Xem acf/pacf để đoán các giá trị ứng viên cho 𝑝 và 𝑞.
+ grid search với 𝑝, 𝑞 nhỏ (giữ 𝑑 cố định) và chọn mô hình theo aic/bic.
+ Chẩn đoán phần dư: kiểm tra residual có gần “white noise” hay không, nhằm đánh giá mô hình đã bắt được cấu trúc chính 
của chuỗi chưa.

In [ ]:
# ============================================================
# QUESTION 3: ARIMA Decision Process
# ============================================================
print("="*60)
print("QUESTION 3: Quy trình ra quyết định ARIMA")
print("="*60)

print("""
📋 QUY TRÌNH CHỌN THAM SỐ ARIMA(p, d, q):

┌──────────────────────────────────────────────────────────────┐
│  STEP 1: QUAN SÁT CHUỖI GỐC                                  │
│  ➜ Nhận diện xu hướng (trend) và mùa vụ (seasonality)        │
│  ➜ Từ biểu đồ Q1: Có mùa vụ theo ngày, không có trend rõ     │
└──────────────────────────────────────────────────────────────┘
           │
           ▼
┌──────────────────────────────────────────────────────────────┐
│  STEP 2: KIỂM ĐỊNH DỪNG (ADF/KPSS) ĐỂ CHỌN d                 │
│  ➜ Nếu ADF p < 0.05: Chuỗi dừng → d = 0                      │
│  ➜ Nếu ADF p >= 0.05: Cần differencing → d = 1               │
│  ➜ Kết quả: d = 0 (chuỗi PM2.5 đã dừng)                      │
└──────────────────────────────────────────────────────────────┘
           │
           ▼
┌──────────────────────────────────────────────────────────────┐
│  STEP 3: XEM ACF/PACF ĐỂ ĐOÁN p VÀ q                         │
│  ➜ PACF cut-off sau lag k → p ≈ k                            │
│  ➜ ACF cut-off sau lag m → q ≈ m                             │
│  ➜ Từ Q1: PACF spike ở 1, 2 → p ∈ {1, 2, 3}                  │
│           ACF decay dần → q ∈ {0, 1, 2}                      │
└──────────────────────────────────────────────────────────────┘
           │
           ▼
┌──────────────────────────────────────────────────────────────┐
│  STEP 4: GRID SEARCH VỚI p, q NHỎ                            │
│  ➜ Giữ d cố định (từ Step 2)                                 │
│  ➜ Thử p ∈ {0,1,2,3}, q ∈ {0,1,2}                            │
│  ➜ Chọn model có AIC/BIC thấp nhất                           │
└──────────────────────────────────────────────────────────────┘
           │
           ▼
┌──────────────────────────────────────────────────────────────┐
│  STEP 5: CHẨN ĐOÁN PHẦN DƯ (RESIDUAL DIAGNOSTICS)            │
│  ➜ Kiểm tra residual có gần white noise không                │
│  ➜ ACF của residual không có spike đáng kể                   │
│  ➜ Ljung-Box test p > 0.05 → residual = white noise          │
│  ➜ Nếu không đạt → quay lại Step 3-4                         │
└──────────────────────────────────────────────────────────────┘
""")

# Demo: Grid Search conceptual
print("\n" + "="*60)
print("📊 Demo Grid Search (conceptual)")
print("="*60)

grid_results = {
    'p': [0, 1, 1, 2, 2, 2],
    'd': [0, 0, 0, 0, 0, 0],
    'q': [1, 0, 1, 0, 1, 2],
    'AIC': [95000, 92000, 91500, 91800, 91200, 91400],
    'BIC': [95100, 92150, 91700, 92000, 91450, 91700]
}
grid_df = pd.DataFrame(grid_results)
grid_df['Config'] = grid_df.apply(lambda r: f"ARIMA({int(r['p'])},{int(r['d'])},{int(r['q'])})", axis=1)

print("\n   Model configurations and their AIC/BIC (example):")
print(grid_df[['Config', 'AIC', 'BIC']].to_string(index=False))
print(f"\n   → Best model: ARIMA(2,0,1) with lowest AIC = 91200")

### 📝 Giải thích chi tiết Question 3 - Quy trình ARIMA:

---

#### **Step 1: Quan sát chuỗi gốc**
- Vẽ time series plot để nhận diện **trend** (xu hướng tăng/giảm dài hạn) và **seasonality** (pattern lặp lại).
- **Kết quả từ Q1:** PM2.5 có mùa vụ theo ngày (24h) và theo mùa (năm), không có trend rõ ràng.

---

#### **Step 2: Kiểm định dừng để chọn d**
- **ADF test (Augmented Dickey-Fuller):**
  - H0: Chuỗi có unit root (không dừng)
  - Nếu p < 0.05: Bác bỏ H0 → d = 0
  - Nếu p >= 0.05: d = 1 (hoặc nhiều hơn)

- **KPSS test:** Bổ sung để confirm (H0 khác với ADF)

- **Kết quả:** PM2.5 dừng → **d = 0**

---

#### **Step 3: Xem ACF/PACF để chọn p, q**

| Quan sát | Gợi ý |
|----------|-------|
| PACF cut-off sau lag k | p ≈ k |
| ACF cut-off sau lag m | q ≈ m |
| ACF giảm dần, PACF spike | AR process (dùng p) |
| ACF spike, PACF giảm dần | MA process (dùng q) |
| Cả hai giảm dần | ARMA (cần cả p và q) |

---

#### **Step 4: Grid Search**
- Không grid search bừa, chỉ thử các giá trị hợp lý từ Step 3
- Metric: **AIC** (Akaike Information Criterion) hoặc **BIC** (Bayesian IC)
- Chọn model có AIC/BIC thấp nhất

---

#### **Step 5: Chẩn đoán phần dư**
- Residual phải gần **white noise** (nhiễu trắng):
  - Mean ≈ 0
  - Variance không đổi
  - ACF của residual không có spike đáng kể
  - Ljung-Box test: p > 0.05

- Nếu residual vẫn có pattern → model chưa capture hết cấu trúc → cần điều chỉnh p, q hoặc thêm seasonal component